In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, fcd, ld):
        super(VAE, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+6, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, fcd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, rest, nback, emoid):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), 
                       rest.unsqueeze(1), nback.unsqueeze(1), emoid.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        return x

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

vae = VAE(34716, 30)
vae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_1000_z30_cov6.torch'))
vae.eval()

print('Done')

Done


In [2]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [3]:
# Load FC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
lowrankdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/Top10/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
no_snps_subs = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/fc_subs_no_snps.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []
subids = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    if sub in no_snps_subs:
        continue
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        subids.append(sub)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

print([a.shape for a in [rest, nback, emoid, race, sex, age]])

[(1155, 34716), (1155, 34716), (1155, 34716), (1155,), (1155,), (1155,)]


In [4]:
# Load small SNPs

snps = []
no_snps_subs = []

snps_file = pickle.load(open('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/snps_all_subs_big.pkl', 'rb'))

for sub in subids:
    if sub not in snps_file:
        print(sub)
        # no_snps_subs.append(sub)
    snps.append(snps_file[sub])

snps = np.stack(snps)
snps[np.isnan(snps)] = 0

print(snps.shape)

(1155, 35621)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

class SNPVAE(nn.Module):
    def __init__(self, snpd, ld):
        super(SNPVAE, self).__init__()
        self.ld = ld
        self.enc1 = nn.Linear(snpd*3, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+2, 1000).float().cuda()
        self.dec20 = nn.Linear(1000, snpd).float().cuda()
        self.dec21 = nn.Linear(1000, snpd).float().cuda()
        self.dec22 = nn.Linear(1000, snpd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)

    def dec(self, z, sex, race):
        z = torch.cat([z, sex.unsqueeze(1), race.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x0 = self.dec20(x)
        x1 = self.dec21(x)
        x2 = self.dec22(x)
        return torch.stack([x0, x1, x2], dim=1)

def latent_loss(z):
    C = z.T@z
    mu = torch.mean(z, dim=0)
    tgt1 = torch.eye(z.shape[-1]).float().cuda()*len(z)/10
    tgt2 = torch.zeros(z.shape[-1]).float().cuda()
    lossC = rmse(C, tgt1)
    lossmu = rmse(mu, tgt2)
    return lossC, lossmu, C, mu

def decor_loss(z, sex, race):
    sex = sex - torch.mean(sex)
    race = race - torch.mean(race)
    sexp = torch.einsum('n,nz->z', sex, z)
    racep = torch.einsum('n,nz->z', race, z)
    tgt = torch.zeros(z.shape[-1]).float().cuda()
    loss_sex = rmse(sexp, tgt)
    loss_race = rmse(racep, tgt)
    return loss_sex + loss_race

ce = nn.CrossEntropyLoss()

# snpvae = SNPVAE(10433, 30)
snpvae = SNPVAE(35621, 100)
snpvae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/PNC/vae_snps_big_1000_z100_cov2.torch'))
snpvae.eval()

xclass = torch.from_numpy(snps).long().cuda()
x = torch.cat([xclass == 0, xclass == 1, xclass == 2], dim=1).float().cuda()
sex_t = torch.from_numpy(sex).float().cuda()
race_t = torch.from_numpy(race).float().cuda()

print('Done')

Done


In [8]:
import scipy.stats as stats

zz = snpvae.enc(x).detach().cpu().numpy()

def corr(z, y):
    z = z - np.mean(z, axis=0, keepdims=True)
    y = y - np.mean(y)
    xx = np.einsum('na,na->a', z, z)
    yy = np.einsum('n,n->', y, y)
    xy = np.einsum('na,n->a', z, y)
    rho = xy/((xx*yy)**0.5)
    n = z.shape[0]
    m = z.shape[1]
    df = n-2
    t = rho*(df/(1-rho**2))**0.5
    t[t < 0] = -t[t < 0]
    p = (1-stats.t.cdf(t, df))*2
    p *= m
    p[p > 1] = 1
    return rho, p

def to_cat(y):
    cats = set()
    for yy in y:
        cats.add(yy)
    cats = list(cats)
    yp = np.zeros(len(y))
    for j,yy in enumerate(y):
        for i,cat in enumerate(cats):
            if yy == cat:
                yp[j] = i
    return yp

demo = pncdemo

n = 0
m = 0
for key in demo.keys():
    z = []
    y = []
    for i,sub in enumerate(subids):
        if sub in demo[key]:
            z.append(zz[i])
            y.append(demo[key][sub])
    if len(z) == 0:
        continue
    n += 1
    nn = len(z)
    z = np.stack(z)
    y = np.array(y)
    if isinstance(y[0], str):
        y = to_cat(y)
    rho, p = corr(z, y)
    if np.any(p < 0.01):
        m += 1
        print(key, 'yes', min(p), nn)
print(n, m)

PEITANG yes 0.004350412149722871 1155
LNB_FP yes 0.00988712738887454 1141
169 2


/tmp/ipykernel_339617/3357025902.py:11: RuntimeWarning: invalid value encountered in divide
  rho = xy/((xx*yy)**0.5)


In [10]:
len(subids)

1155

In [11]:
z.shape

AttributeError: 'list' object has no attribute 'shape'